<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>Information Processing</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from typing import List, Optional


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import langchain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain, create_extraction_chain

import langchain_core
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

import langchain_community
from langchain_community.document_loaders import WebBaseLoader

import langchain_openai
from langchain_openai import ChatOpenAI, OpenAI

import langchain_text_splitters
from langchain_text_splitters import CharacterTextSplitter


import watermark

%load_ext watermark
%matplotlib inline

/Users/bgoncalves/My Drive/DataForScience/LangChain/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.


We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 24f5062fbf46a87bfe9be08eb40e50ecbf9f4e00

langchain               : 0.3.25
pandas                  : 2.2.3
numpy                   : 2.2.5
langchain_core          : 0.3.62
langchain_text_splitters: 0.3.8
watermark               : 2.5.0
langchain_openai        : 0.3.18
langchain_community     : 0.3.24
matplotlib              : 3.10.3
pydantic                : 2.11.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# Text Summarization

## Summarizing a Paragraph

In [4]:
prompt_template = """Write a concise summary of the following:

"{text}"

CONCISE SUMMARY:"""

In [5]:
text = """The history of Portugal can be traced from circa 400,001 years ago, when the region of present-day Portugal was inhabited by Homo heidelbergensis.

The Roman conquest of the Iberian Peninsula, which lasted almost two centuries, led to the establishment of the provinces of Lusitania in the south and Gallaecia in the north of what is now Portugal. Following the fall of Rome, Germanic tribes controlled the territory between the 5th and 8th centuries, including the Kingdom of the Suebi centred in Braga and the Visigothic Kingdom in the south.

The 711–716 invasion by the Islamic Umayyad Caliphate conquered the Visigoth Kingdom and founded the Islamic State of Al-Andalus, gradually advancing through Iberia. In 1095, Portugal broke away from the Kingdom of Galicia. Afonso Henriques, son of the count Henry of Burgundy, proclaimed himself king of Portugal in 1139. The Algarve (the southernmost province of Portugal) was conquered from the Moors in 1249, and in 1255 Lisbon became the capital. Portugal's land boundaries have remained almost unchanged since then. During the reign of King John I, the Portuguese defeated the Castilians in a war over the throne (1385) and established a political alliance with England (by the Treaty of Windsor in 1386).

From the late Middle Ages, in the 15th and 16th centuries, Portugal ascended to the status of a world power during Europe's "Age of Discovery" as it built up a vast empire. Signs of military decline began with the Battle of Alcácer Quibir in Morocco in 1578; this defeat led to the death of King Sebastian and the imprisonment of much of the high nobility, which had to be ransomed at great cost. This eventually led to a small interruption in Portugal's 800-year-old independence by way of a 60-year dynastic union with Spain between 1580 and the beginning of the Portuguese Restoration War led by John IV in 1640. Spain's disastrous defeat in its attempt to conquer England in 1588 by means of the Invincible Armada was also a factor, as Portugal had to contribute ships for the invasion. Further setbacks included the destruction of much of its capital city in an earthquake in 1755, occupation during the Napoleonic Wars, and the loss of its largest colony, Brazil, in 1822. From the middle of the 19th century to the late 1950s, nearly two million Portuguese left Portugal to live in Brazil and the United States.[1]

In 1910, a revolution deposed the monarchy. A military coup in 1926 installed a dictatorship that remained until another coup in 1974. The new government instituted sweeping democratic reforms and granted independence to all of Portugal's African colonies in 1975. Portugal is a founding member of NATO, the Organisation for Economic Co-operation and Development (OECD), the European Free Trade Association (EFTA), and the Community of Portuguese Language Countries. It entered the European Economic Community (now the European Union) in 1986. 
"""

In [6]:
summary_prompt = PromptTemplate.from_template(prompt_template).invoke(text)

In [7]:
print(summary_prompt.text)

Write a concise summary of the following:

"The history of Portugal can be traced from circa 400,001 years ago, when the region of present-day Portugal was inhabited by Homo heidelbergensis.

The Roman conquest of the Iberian Peninsula, which lasted almost two centuries, led to the establishment of the provinces of Lusitania in the south and Gallaecia in the north of what is now Portugal. Following the fall of Rome, Germanic tribes controlled the territory between the 5th and 8th centuries, including the Kingdom of the Suebi centred in Braga and the Visigothic Kingdom in the south.

The 711–716 invasion by the Islamic Umayyad Caliphate conquered the Visigoth Kingdom and founded the Islamic State of Al-Andalus, gradually advancing through Iberia. In 1095, Portugal broke away from the Kingdom of Galicia. Afonso Henriques, son of the count Henry of Burgundy, proclaimed himself king of Portugal in 1139. The Algarve (the southernmost province of Portugal) was conquered from the Moors in 124

In [8]:
llm = OpenAI(temperature=0)

In [9]:
output = llm.invoke(summary_prompt.text)
print (output)

 Portugal's history dates back to 400,001 years ago when it was inhabited by Homo heidelbergensis. It was later conquered by the Romans and then controlled by Germanic tribes. In 711, it was invaded by the Islamic Umayyad Caliphate and eventually broke away from the Kingdom of Galicia in 1095. Portugal became a world power during the Age of Discovery in the 15th and 16th centuries, but suffered setbacks such as defeat in the Battle of Alcácer Quibir and the loss of its largest colony, Brazil. In 1910, a revolution deposed the monarchy and a dictatorship was installed until 1974. Portugal granted independence to its African colonies in 1975 and is a member of various international organizations. It joined the European Economic Community in 1986.


## Summarizing a Document

In [10]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [11]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

In [12]:
print(docs[0].page_content[2000:4000])

uding current information, code execution capability, access to proprietary information sources and more.





Overview of a LLM-powered autonomous agent system.

Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) 

In [13]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff")

In [14]:
chain

StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11154ecf0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11154ee40>, root_client=<openai.OpenAI object at 0x11158e350>, root_async_client=<openai.AsyncOpenAI object at 0x11158ee90>, model_name='gpt-3.5-turbo-1106', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='text')

In [15]:
result = chain.invoke(docs)

print(result["output_text"])

The article discusses the concept of LLM-powered autonomous agents, which use large language models as their core controllers. It covers the components of these agents, including planning, memory, and tool use, as well as case studies and proof-of-concept examples. The challenges and limitations of using natural language interfaces for these agents are also addressed. The article provides citations and references for further reading.


In [16]:
# Define prompt
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.invoke(docs)["output_text"])

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_47936/570235395.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_47936/570235395.py:9: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")


The article discusses the concept of building autonomous agents powered by Large Language Models (LLMs) like GPT. It covers components such as planning, memory, and tool use, with examples like AutoGPT and GPT-Engineer. Challenges include limited context length, planning difficulties, and reliability of natural language interfaces. The article also references various studies and projects in the field of LLM-powered autonomous agents.


## Using MapReduce

In [17]:
llm = ChatOpenAI(temperature=0.7)

### Map

In [18]:
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""

In [19]:
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

### Reduce

In [20]:
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""

In [21]:
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

Take a list of documents, combines them into a single string, and passes this to an LLMChain

In [22]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

Iteratively reduces the mapped documents

In [23]:
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_47936/1743346405.py:1: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  reduce_documents_chain = ReduceDocumentsChain(


Full MapReduce chain

In [24]:
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_47936/2003783529.py:1: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  map_reduce_chain = MapReduceDocumentsChain(


In [25]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000, 
    chunk_overlap = 0
)

split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [26]:
type(split_docs[0])

langchain_core.documents.base.Document

In [27]:
print(len(split_docs))

13


In [28]:
print(split_docs[10].page_content)

Conversatin samples:
[
  {
    "role": "system",
    "content": "You will get instructions for code to write.\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\nMake sure that every detail of the architecture is, in the end, implemented as code.\n\nThink step by step and reason yourself to the right decisions to make sure we get it right.\nYou will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.\n\nThen you will output the content of each file including ALL code.\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such that\nFILENAME is the lowercase file name including the file extension,\nLANG is the markup code block language for the code's language, and CODE is the code:\n\nFILENAME\n```LANG\nCODE\n```\n\nYou will start with the \"entrypoint\" file, then go to the ones that a

# Structured Information Extraction

Based on https://python.langchain.com/v0.1/docs/use_cases/extraction/quickstart/

We start by loading a number of tweets from a csv file

In [29]:
data = pd.read_csv('data/trump.csv')
data.head()

text           created_at  \
0  .@FoxNews is no longer the same. We miss the g...  05-19-2020 01:59:49   
1  So the so-called HHS Whistleblower was against...  05-18-2020 14:44:21   
2  .....mixed about even wanting us to get out. T...  05-18-2020 14:39:40   
3  Wow! The Front Page @washingtonpost Headline r...  05-18-2020 12:47:40   
4  MAGA crowds are bigger than ever! https://t.co...  05-18-2020 12:26:37   

                id_str  
0  1262563582086184970  
1  1262393595560067073  
2  1262392415513690112  
3  1262364231288197123  
4  1262358931982123008

In [30]:
data.shape

(2834, 3)

And defining the data structures that will hold the data we want our chain to extract. The comments help the LLM understand what each field means

In [31]:
class Person(BaseModel):
    """Information about a person."""
    name: Optional[str] = Field(default=None, description="The name of the person")
    twitter_handle: Optional[str] = Field(
        default=None, description="The twitter handle if known"
    )

class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert data extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

and our chain

In [33]:
runnable = prompt | llm.with_structured_output(schema=Data)

/Users/bgoncalves/My Drive/DataForScience/LangChain/.venv/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1656: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/Users/bgoncalves/My Drive/DataForScience/LangChain/.venv/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1669: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [34]:
for i, tweet in enumerate(data['text'].head(10)):
    response = runnable.invoke({"text": tweet})
    print(i, '==>', tweet, "<===", response)

0 ==> .@FoxNews is no longer the same. We miss the great Roger Ailes. You have more anti-Trump people by far than ever before. Looking for a new outlet! https://t.co/jXxsF0flUM <=== people=[Person(name='Roger Ailes', twitter_handle=None)]
1 ==> So the so-called HHS Whistleblower was against HYDROXYCHLOROQUINE. Then why did he make and sign an emergency use authorization? @NorahODonnell said “He shared his concerns with a reporter.” In other words he LEAKED. A dumb @60Minutes hit job on a grandstanding Never Trumper! <=== people=[Person(name='HHS Whistleblower', twitter_handle='NorahODonnell'), Person(name='60Minutes', twitter_handle=None)]
2 ==> .....mixed about even wanting us to get out. They make a fortune $$$ by having us stay and except at the beginning we never really fought to win. We are more of a police force than the mighty military that we are especially now as rebuilt. No I am not acting impulsively! <=== people=[Person(name='unknown', twitter_handle=None)]
3 ==> Wow! The F

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>